## 무선청소기 모델별  비교분석 
 - 여러 제조사별 무선 청소기의 가격과 성능에 대한 데이터를 수집하고, 
   이를 분석하는 과정을 통해 각자의 상황에 맞는 제품군이 무엇인지 분석한다.

### 데이터 수집

#### 1. 한 페이지 크롤링
  - 한 페이지에 접속해서 정보를 가져오는 과정을 실습한다.
  - 상품에 대한 가격비교 사이트인 다나와에서는 제품별로 용량, 크기 같은 조건을
    설정해서 검색할 수 있다.

  - 다나와를 활용해 무선청소기 제품의 가격과 스펙 목록을 수집한다.

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
#selenium으로 다나와 검색 결과 URL에 접속
driver = webdriver.Chrome('c:/Study/Python/datadown/chromedriver.exe')
driver.get("http://search.danawa.com/dsearch.php?query=무선청소기&tab=main")

#웹 페이지의 HTML 정보 가져오기
html=driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [14]:
#페이지에 대한 무선청소기 정보 가져오기
search_result=soup.select('div.main_prodlist.main_prodlist_list > ul.product_list > li.prod_item')
print(len(search_result))
#search_result[0]

44


In [15]:
#상품명 가져오기 -1
#p태그의 class이름이 prod_name인 첫번째 p태그요소와 직계 후손 a태그요소를 가져온다.

title = search_result[0].select('p.prod_name > a')[0].get_text().strip()
title

'LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571'

In [16]:
#상품명 가져오기 -2
#직계 후손을 가져오려면: 조상태그 > 직계후손 > 다음 직계후손 > 그다음 직계후손
#위와 같이 띄어쓰기와 꺽쇠'>'를 표기해야한다

title = search_result[0].select('div.prod_main_info > div.prod_info > p.prod_name > a')[0].get_text().strip()
title

'LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571'

In [17]:
#상품명 가져오기 -3
#직계가 아닌 먼 후손을 가져오려면 꺽쇠'>'없이 띄어쓰기만 한다.
#아래를 보면 div.prod_main_info와 p.prod_name사이에 한칸 띄어쓰기가 있다.

title = search_result[0].select('div.prod_main_info p.prod_name > a')[0].get_text().strip()
title

'LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571'

In [9]:
#스펙 목록 정보 가져오기 -1

specs = search_result[0].select('div.prod_main_info > div.prod_info > dl.prod_spec_set > dd > div.spec_list')
#specs

In [7]:
#스펙 목록 정보 가져오기-2

specs = search_result[0].select('div.spec_list')
#specs

In [29]:
#상품 가격 가져오기
price = search_result[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
price

'1203650'

첫 페이지의 상품 정보를 수집하는 반복문을 실행해보자.
(검색결과 페이지에서 상품 리스트 중간중간에 상품이 아닌 광고가 있는 경우에는 지정한 태그 요소를 불러올 수 없을 수 있다. 그래서 try/except문을 사용한다.)

검색결과를 조회하는 사이트에서 반복문으로 아이템 정보를 추출해올때 try/except문을 사용하는것이 유용하다.

In [18]:
#반복문으로 검색 결과의 1페이지에 대한 상품 정보 추출
prod_data = []

for item in search_result:
    try:
        title = item.select('p.prod_name > a')[0].text.strip()
    except:
        title = ""
    try:
        specs = item.select('div_spec_list')[0].text.strip()
    except:
        spec_list = ""
    try:
        price = item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
    except:
        price=0
    prod_data.append([title, specs, price])

print(len(prod_data)) #44개의 상품정보를 가져왔다.

# for x in prod_data:
#     print(x)

44


#### 2. 여러 페이지 크롤링


**url을 제어해서 여러 페이지 자동으로 이동해서 원하는 정보를 추출하는 함수 생성해보자**


상품 검색 결과가 여러 페이지에 걸쳐서 나오기 때문에 지금까지 진행한 한 페이지에 대한 검색결과를 수집하는 코드를 함수로 만든다.

In [19]:
#상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for item in search_result:
        try:
            title = item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ""
        try:
            specs = item.select('div.spec_list')[0].text.strip()
        except:
            specs = ""
        try:
            price = item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
        except:
            price=0
        prod_data.append([title, specs, price])
    return prod_data

- 위에서 만든 함수와 반복문을 이용해 전체 페이지의 상품 정보 데이터를 수집한다.

1. 먼저 검색결과의 첫페이지에서 다른 페이지로 이동하면 변경되는 URL을 찾는다.

2. 반복문에서 사용하기 위해 검색어와 페이지가 URL에서 들어가는 위치를 파악한다.

3. 검색어와 페이지 정보만 변경해서 다음과 같이 URL 만들어서 웹 브라우저로 페이지를 열어서 호출하여 여러페이지를 가져온다
  
주의:: 배포된 url은 아래와같다. 그러나 실제 웹사이트에서 가져온 url과 다를수도 있기때문에 직접 여러 페이지를 조회해서 살펴봐야한다.

http://search.danawa.com/dsearch.php?query={검색어}&originalQuery={검색어}&volumeType=allvs&page={페이지}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods

함수 매개변수로 keyword와 page를 입력받아 format 함수를 이용해 문자열을 조합해서 검색 URL을 만드는 식으로 함수를 만든다

In [32]:
#다나와 검색 URL을 만들어주는 함수 '\'는 한문장을 여러줄로 보여준다.
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&originalQuery={}&previousKeyword={}&volumeType=allvs&page={}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=103740&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2490&defaultVaTab=153544&tab=goods'.format(keyword, keyword, keyword, page)

In [33]:
keyword = '무선청소기'
page = 5
url = get_search_page_url(keyword, page)
print(url)

http://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&volumeType=allvs&page=5&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=103740&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2490&defaultVaTab=153544&tab=goods


#### tqdm 라이브러리 활용 (진행현황 보여주기)

- tqdm라이브러리를 이용하면 현재 진행 상황을 표시할 수 있다.0%(시작)부터 100%(완료)까지 bar graph로 progress 현황을 보여준다.
(검색시 시간이 많이 걸릴경우 현재 작업 진행현황을 볼 수 있도록 하는 tqdm사용한다.)


- 참고: jpuyternotebook에서는 해당 ipynb파일을 완전히 닫고 열면서 kernel이 restart되면, widget state를 찾을 수 없어서 tqdm을 통해 progress를 보여주는 bar graph가 display되지 않을 수 있다. kernel 생성 및 다시 실행을 하면 보여진다.

**참고: tqdm 설치방법**

1. 원도 시작메뉴 -> Anaconda Prompt 관리자 권한으로 실행

2. Prompt윈도우에서 아래와 같이 입력:

(base) C:\WINDOWS\system32>activate pydata
(pydata) C:\WINDOWS\system32>

 conda install -n base -c conda-forge widgetsnbextension
 
 conda install -n pydata -c conda-forge ipywidgets
 
3. jupyter notebook 에서  !pip install tqdm  설치
4. Kernel Restart 한다.

In [34]:
!pip install tqdm

In [34]:
import time
from tqdm.notebook import tqdm

total_page = 10

#페이지 이동할때 아래와같이 page가 1부터 최대페이지까지 이동하는거을 simulate해본다
for page in tqdm(range(1, total_page+1)):
    time.sleep(5) #페이지 이동시 5초대기

  0%|          | 0/10 [00:00<?, ?it/s]

상품 검색 결과에서 여러페이지에 걸친 상품 정보 수집을 진행해본다.

한 페이지의 상품정보를 수집하는 함수 만들기, URL분석을 통해서 원하는 페이지로 이동하는 함수 만들기 등을 이용해서 여러페이지에 걸친 '무선청소기' 검색 결과를 수집한다.

In [16]:
#다시 driver생성 및 웹페이지 불러오기 실행한다
driver = webdriver.Chrome('c:/Study/Python/datadown/chromedriver.exe')
driver.implicitly_wait(3)

#검색 키워드, 검색할 페이지 최대수, 검색결과를 답을 리스트 생성
keyword = '무선청소기'
total_page = 10
prod_data_total = []

In [17]:
for page in tqdm(range(1, total_page+1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    search_result=soup.select('div.main_prodlist.main_prodlist_list > ul.product_list > li.prod_item')
    
    prod_item_list = get_prod_items(search_result)
    prod_data_total = prod_data_total + prod_item_list

  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
#1부터 최대페이지까지 조회애서 추출해온 prod_data_total 리스트를 dataframe에 저장
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns=['상품명', '스펙 목록', '가격']
data.head()

,상품명,스펙 목록,가격
0,LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 흡입+걸레겸용 / [성능] 2중터보싸...,1203710
1,샤오미 CLEANFLY 차량용 무선 청소기 3세대 (해외구매),"차량용청소기 / 무선 / 흡입력: 16,800Pa / 최대출력: 120W / 헤파필...",50080
2,7WCkV4yUtk,7WCkV4yUtk371220,371220
3,샤오미 미지아 차량용 핸디 무선 청소기 (해외구매),"차량용청소기 / 무선 / 흡입력: 13,000Pa / 최대출력: 120W / 헤파필...",36230
4,LG전자 코드제로 A9 A9100S,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 흡입+걸레겸용 / [성능] 2중터보싸...,462570


In [21]:
#dataframe을 excel파일에 저장
filename='../data3/{}_p1-p{}_Danawa_searchResult.xlsx'.format(keyword, total_page)
data.to_excel(filename, index=False)

driver.close()